In [1]:
import pickle as pkl
import re 

path = '/home/liuxiao/TuringGithub/XiaoData/ZeroShotJudge/chatgpt_free_viewing_zeroshot_judge.pkl'
with open(path,'rb') as f:
    judge_results = pkl.load(f)

In [2]:
judge_results

[{'imageID': '001',
  'scanpath': [[660, 512],
   [920, 752],
   [1180, 608],
   [1100, 576],
   [1260, 624],
   [1260, 704],
   [1100, 688],
   [820, 512],
   [500, 512],
   [460, 624],
   [380, 672],
   [500, 560],
   [460, 656],
   [140, 560],
   [40, 512]],
  'groundtruth': 'machine',
  'human_groundtruth': '',
  'machine_groundtruth': 'ivsn',
  'raw_response': 'Answer: Human\n\nJustification: The scanpath demonstrates a broad and organic coverage of the image, moving fluidly from the center outward and sweeping both the right and left sides. There is a clear left-to-right and then right-to-left exploration, which is common in human visual behavior as they attempt to make sense of a visually complex scene like this pile of colorful crushed cans. The attention does not fixate repetitively on one region or show unnatural jumps, which are more typical for AI-generated patterns. Instead, it reflects curiosity and a scanning strategy that aligns with human tendencies in a free-viewing t

In [27]:
import re

# special case '0525.jpg'

def findanswer(text):
#     templates = [r'(?:\*\*Caption\s*(\d)|^(\d)\. \:)',
#                  r'(?:\#\#\#\sCaption\s*(\d)|^(\d)\. \:)',
#                  r'(?:\*\**(\d)|^(\d)\. \:)',
#                  r'(?:\#\#\#\s(\d)|^(\d)\. \:)',
#                  r'(?:(\d)\. \*\*)']
    results = []
#     print(re.search(templates[0],text))
    template = r'(?:.{0,5}Justification)'

    print(re.search(template,text))
    if re.search(template,text) is not None:
        summary_text = text[0:re.search(template,text).start()]
    else:
        summary_text = text
    summary_text = summary_text.split('\n')[0]
    print(summary_text)
    results = []
    for i, m in enumerate(re.finditer('(?:(AI-generated)|(Human))',summary_text)):
        print(m.group(0))
        if 'AI-generated' in m.group(0):
            results.append('machine')
        elif 'Human' in m.group(0):
            results.append('human')
        else:
            assert False
    print(results)
    if len(results) != 1 or len([_ for _ in results if _ in ['human','machine']]) != 1:
        assert False
    return results
    
judge_refined_results = []
for d in judge_results:
    
    print('==='*20)
    print(d['imageID'])
#     if d['imageID'] not in judge_refined_results:
    result = findanswer(d['raw_response'])
#         results_d[k] = results
    d['response_speaker'] = result
    judge_refined_results.append(d)
    

001
<re.Match object; span=(15, 28), match='Justification'>
Answer: Human
Human
['human']
001
None
**Answer: Human**  
Human
['human']
001
<re.Match object; span=(22, 35), match='Justification'>
Answer: AI-generated
AI-generated
['machine']
001
<re.Match object; span=(22, 35), match='Justification'>
Answer: AI-generated
AI-generated
['machine']
001
<re.Match object; span=(22, 35), match='Justification'>
Answer: AI-generated
AI-generated
['machine']
001
<re.Match object; span=(15, 28), match='Justification'>
Answer: Human
Human
['human']
001
<re.Match object; span=(22, 35), match='Justification'>
Answer: AI-generated
AI-generated
['machine']
001
<re.Match object; span=(23, 36), match='Justification'>
Answer: AI-generated  
AI-generated
['machine']
001
<re.Match object; span=(15, 28), match='Justification'>
Answer: Human
Human
['human']
001
<re.Match object; span=(15, 28), match='Justification'>
Answer: Human
Human
['human']
001
<re.Match object; span=(15, 28), match='Justification'>
Ans

['machine']
087
<re.Match object; span=(15, 28), match='Justification'>
Answer: Human
Human
['human']
087
<re.Match object; span=(15, 28), match='Justification'>
Answer: Human
Human
['human']
087
<re.Match object; span=(15, 28), match='Justification'>
Answer: Human
Human
['human']
087
<re.Match object; span=(15, 28), match='Justification'>
Answer: Human
Human
['human']
087
<re.Match object; span=(22, 35), match='Justification'>
Answer: AI-generated
AI-generated
['machine']
087
<re.Match object; span=(15, 28), match='Justification'>
Answer: Human
Human
['human']
087
<re.Match object; span=(15, 28), match='Justification'>
Answer: Human
Human
['human']
088
<re.Match object; span=(15, 28), match='Justification'>
Answer: Human
Human
['human']
088
<re.Match object; span=(15, 28), match='Justification'>
Answer: Human
Human
['human']
088
<re.Match object; span=(19, 34), match='**Justification'>
**Answer**: Human
Human
['human']
088
<re.Match object; span=(22, 35), match='Justification'>
Answer

160
<re.Match object; span=(19, 34), match='**Justification'>
**Answer: Human**
Human
['human']
160
<re.Match object; span=(22, 35), match='Justification'>
Answer: AI-generated
AI-generated
['machine']
160
<re.Match object; span=(15, 28), match='Justification'>
Answer: Human
Human
['human']
160
<re.Match object; span=(16, 29), match='Justification'>
Answer: Human  
Human
['human']
160
<re.Match object; span=(15, 28), match='Justification'>
Answer: Human
Human
['human']
161
<re.Match object; span=(22, 35), match='Justification'>
Answer: AI-generated
AI-generated
['machine']
161
<re.Match object; span=(15, 28), match='Justification'>
Answer: Human
Human
['human']
161
<re.Match object; span=(22, 35), match='Justification'>
Answer: AI-generated
AI-generated
['machine']
161
<re.Match object; span=(22, 35), match='Justification'>
Answer: AI-generated
AI-generated
['machine']
161
<re.Match object; span=(15, 28), match='Justification'>
Answer: Human
Human
['human']
161
<re.Match object; span=(

### Compile Trials

In [40]:
import numpy as np
import json
trial_count = 1
# label_dict = {'human':0,'machine':1,'blip':1,'microsoft_api':1,'ofa':1,'git':1,'clipcap':1,'catch':1}
label_dict = {'human':0,'machine':1,'Human':0,'Machine':1,'azure':1,'detectron':1,'google':1,'catch':1,'rekognition':1}
modelname_dict= {'':'','ivsn':'IVSN','gbvs':'GBVS','deepgaze':'DeepGazeIII','chatgpt':"ChatGPT",'swap':'SwappedBackground','catch':'CatchTrial'}

def construct_confmat():
    labels = ['h_ivsn','h_gbvs','h_deepgaze']#,'h_catch']
    confmatrices = {**{'h_machine':np.zeros((2,2))}, **{label: np.zeros((1,2)) for label in labels}}
    return confmatrices

ai_judge = {}
confmat_overall = construct_confmat()
for d in judge_refined_results:
    trialdata = {'response_speaker':d['response_speaker'][0],
                    'groundtruth':d['groundtruth'],
                    'machine_groundtruth':modelname_dict[d['machine_groundtruth']]}
    ai_judge['Trial_{}'.format(trial_count)] = {'trialData':trialdata}
    trial_count += 1
        
#     confmat_overall['h_machine'][label_dict[trialdata['groundtruth']]][label_dict[trialdata['response_speaker']]] += 1
        
#     if trialdata['groundtruth'] == 'machine':
#         confmat_overall['h_{}'.format(trialdata['machine_groundtruth'])][0][label_dict[trialdata['response_speaker']]] += 1          
            
            
    

In [41]:
json.dump(ai_judge, open("/home/liuxiao/TuringGithubRunnable/attention_prediction_task/Plot/Data_all/ai_judge_zeroshot_chatgpt_fv.json","w"))
json.dump(ai_judge, open("/home/liuxiao/TuringGithub/XiaoData/all_results/ai_judge_zeroshot_chatgpt_fv.json","w"))


In [37]:
confmat_overall

{'h_machine': array([[2096.,  304.],
        [ 435.,  285.]]),
 'h_ivsn': array([[171.,  69.]]),
 'h_gbvs': array([[220.,  20.]]),
 'h_deepgaze': array([[ 44., 196.]])}